In [43]:
import numpy as np

def run_graph_coloring_dsb(adj_matrix, num_colors, steps=1000, a0=1.0, c0=1.0, dt=0.01):
    """
    Run a Dynamic Simulation-Based (DSB) algorithm for solving graph coloring problems.
    
    Args:
        adj_matrix: Adjacency matrix of the graph (1 if vertices are connected, 0 otherwise)
        num_colors: Maximum number of colors to use
        steps: Number of simulation steps
        a0: System parameter (constant) controlling the momentum
        c0: Coupling strength for graph coloring constraints
        dt: Time step size
    
    Returns:
        color_assignment: The final color assignment (1 to num_colors) for each vertex
        violation_count: Number of edge violations in the final solution
        x_history: History of position values during simulation
    """
    # Get problem size
    N = adj_matrix.shape[0]  # Number of vertices
    K = num_colors  # Number of colors
    
    # Initialize positions and momenta
    # Instead of N binary variables, we now have N×K variables (one for each vertex-color pair)
    x = np.random.uniform(-0.1, 0.1, (N, K))  # Position variables
    y = np.zeros((N, K))                      # Momentum variables
    
    # Initialize history arrays
    x_history = np.zeros((steps + 1, N, K))
    x_history[0] = x.copy()
    
    # Define a(t) function - linear increase from 0 to a0
    def a_t_func(t):
        return min(a0 * t / (0.2 * steps * dt), a0)
    
    # Main simulation loop
    for step in range(1, steps + 1):
        t = step * dt
        a_t = a_t_func(t)
        
        # Get current approximate binary state using sigmoid
        x_prob = 1 / (1 + np.exp(-x))  # Values between 0 and 1
        
        # Calculate the gradient of the energy function
        gradient = np.zeros((N, K))
        
        # 1. Graph coloring constraint: adjacent vertices should have different colors
        for i in range(N):
            for j in range(N):
                if adj_matrix[i, j] == 1:  # If vertices i and j are adjacent
                    for c in range(K):
                        # If j has color c, push i away from color c
                        gradient[i, c] += c0 * x_prob[j, c]
        
        # 2. One-hot encoding constraint: each vertex should have exactly one color
        constraint_weight = 2.0 * c0  # Weight for one-hot encoding constraint
        for i in range(N):
            color_sum = np.sum(x_prob[i])
            for c in range(K):
                # Penalize if sum != 1
                gradient[i, c] += constraint_weight * (color_sum - 1) * x_prob[i, c]
        
        # First part of symplectic Euler: update momenta
        # ẏ = -[a0 - a(t)]x - gradient
        y -= dt * ((a0 - a_t) * x + gradient)
        
        # Second part: update positions
        # ẋ = a0 * y
        x += dt * a0 * y
        
        # Apply inelastic walls to help converge to binary solutions
        for i in range(N):
            # Find the maximum value for this vertex
            max_idx = np.argmax(x[i])
            max_val = x[i, max_idx]
            
            # If max is above threshold, make the solution more "binary"
            if max_val > 3.0:
                # Reset all components
                y[i] = np.zeros(K)
                x[i] = np.ones(K) * -2.0  # All start negative
                x[i, max_idx] = 2.0  # Except the max one
        
        # Store current positions
        x_history[step] = x.copy()
    
    # Get final color assignment
    # For each vertex, select the color with the highest value
    color_assignment = np.argmax(x, axis=1) + 1  # +1 to make colors 1-indexed
    
    # Count violations (adjacent vertices with the same color)
    violation_count = 0
    for i in range(N):
        for j in range(i+1, N):
            if adj_matrix[i, j] == 1 and color_assignment[i] == color_assignment[j]:
                violation_count += 1
    
    return color_assignment, violation_count, x_history

# Example usage
def example_graph_coloring():
    """
    Example of using the DSB algorithm to solve a graph coloring problem.
    """
    # Create a simple graph (pentagon)
    # N = 5
    # adj_matrix = np.zeros((N, N))
    # for i in range(N):
    #     adj_matrix[i, (i+1) % N] = 1
    #     adj_matrix[(i+1) % N, i] = 1

    adj_matrix = np.array([[0, 1, 1, 1],
                       [1, 0, 0, 0],
                       [1, 0, 0, 0],
                       [1, 0, 0, 0]])
    
    # Run the graph coloring algorithm
    num_colors = 2
    color_assignment, violations, _ = run_graph_coloring_dsb(
        adj_matrix, num_colors, steps=2000, a0=1.0, c0=2.0, dt=0.01
    )
    
    print(f"Graph coloring solution for a pentagon:")
    print(f"Color assignment: {color_assignment}")
    print(f"Number of violations: {violations}")
    if violations == 0:
        print("Valid coloring found!")
    else:
        print("No valid coloring found with the given parameters.")
    
    return color_assignment, violations


example_graph_coloring()

Graph coloring solution for a pentagon:
Color assignment: [2 1 1 1]
Number of violations: 0
Valid coloring found!


(array([2, 1, 1, 1]), 0)